In [31]:
import pandas as pd
from prophet import Prophet
import plotly.express as px
from datetime import datetime, timedelta
import yahoo_fin.stock_info as si

# Function to fetch historical data for a stock based on today's date
def fetch_historical_data(ticker, start_date):
    try:
        end_date = datetime.today().strftime('%Y-%m-%d')
        data = si.get_data(ticker, start_date=start_date, end_date=end_date)
        data = data.reset_index()
        data = data[['index', 'close']]  # Ensure only needed columns are included
        data.rename(columns={'index': 'Date', 'close': 'Close'}, inplace=True)

        # Compute Close Shift
        data['Close Shift'] = data['Close'].diff()

        return data

    except Exception as e:
        print(f"Error fetching data: {e}")
        return None

def display_forecasts_from_data(df, ticker, periods=30):
    if df.empty:
        print("Error: Empty DataFrame")
        return

    # Rename columns to match Prophet's expected format and use 'Close Shift' as target
    df.rename(columns={'Date': 'ds', 'Close Shift': 'y'}, inplace=True)

    # Drop any rows with NaN values (which will be the first row after diff())
    df.dropna(inplace=True)

    model = Prophet()
    model.fit(df)

    # Generate future DataFrame starting from the last date +1
    last_date = df['ds'].iloc[-1]
    future_dates = pd.date_range(start=last_date + timedelta(days=1), periods=periods, freq='D')
    future = pd.DataFrame({'ds': future_dates})

    forecast = model.predict(future)

    # Add forecasted close shifts to the last known close price to predict actual prices
    last_known_close = df['Close'].iloc[-1]  # Use 'Close' from the original DataFrame before renaming
    forecast['Forecasted Close'] = last_known_close + forecast['yhat'].cumsum()

    # Print and plot the results
    print("Forecasted Close Prices:")
    print(forecast[['ds', 'Forecasted Close']])

    fig1 = px.line(forecast, x='ds', y='Forecasted Close', labels={'Forecasted Close': 'Forecast'}, title=f'{ticker} Forecast')
    fig2 = px.line(df, x='ds', y='Close', labels={'Close': 'Historical Data'}, title=f'{ticker} Historical Data')

    fig1.show()
    fig2.show()

# Example usage
ticker = 'TSLA'
start_date = datetime.today() - timedelta(days=3650)  # 10 years ago

try:
    historical_data = fetch_historical_data(ticker, start_date)
    if historical_data is not None:
        display_forecasts_from_data(historical_data, ticker)
except Exception as e:
    print(f"Error processing forecasts: {e}")

00:03:50 - cmdstanpy - INFO - Chain [1] start processing
00:03:50 - cmdstanpy - INFO - Chain [1] done processing


Forecasted Close Prices:
           ds  Forecasted Close
0  2024-04-20        146.163288
1  2024-04-21        145.252188
2  2024-04-22        144.952653
3  2024-04-23        144.046165
4  2024-04-24        143.058586
5  2024-04-25        141.646622
6  2024-04-26        140.394801
7  2024-04-27        139.301873
8  2024-04-28        138.176141
9  2024-04-29        137.656667
10 2024-04-30        136.529173
11 2024-05-01        135.324771
12 2024-05-02        133.706257
13 2024-05-03        132.264896
14 2024-05-04        131.006596
15 2024-05-05        129.747003
16 2024-05-06        129.132444
17 2024-05-07        127.955587
18 2024-05-08        126.753904
19 2024-05-09        125.195743
20 2024-05-10        123.876881
21 2024-05-11        122.806528
22 2024-05-12        121.802249
23 2024-05-13        121.510807
24 2024-05-14        120.723738
25 2024-05-15        119.975800
26 2024-05-16        118.931052
27 2024-05-17        118.179486
28 2024-05-18        117.723133
29 2024-05-19  

In [3]:
import pandas as pd
from prophet import Prophet
import plotly.express as px
from IPython.display import display
from datetime import datetime, timedelta

# Function to perform forecasting and plot results
def display_forecasts(asset_csv, periods=30):
    # Define the relative path to the data file
    relative_path = '../Data Collection & Processing/top25_SP500/' + asset_csv
    
    # Load historical data from a CSV file using the relative path
    df = pd.read_csv(relative_path)
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.rename(columns={'Date': 'ds', 'Close': 'y'})
    df = df[['ds', 'y']]

    # Create and train the Prophet model
    model = Prophet(daily_seasonality=True)
    model.fit(df)

    # Generate future dataframe starting from today's date
    today_date = datetime.today().strftime('%Y-%m-%d')
    last_date = df['ds'].max()
    future = pd.DataFrame({'ds': pd.date_range(start=last_date, periods=periods, freq='D')})
    future = future.append(pd.DataFrame({'ds': pd.date_range(start=today_date, periods=periods, freq='D')}))
    
    # Make the forecast
    forecast = model.predict(future)

    # Print out the predicted values from today's date
    print("Predicted Values from Today's Date:")
    print(forecast[['ds', 'yhat']])

    # Plot the forecasts
    fig1 = px.line(forecast, x='ds', y='yhat', labels={'yhat': 'Forecast'}, title=f'{asset_csv} Forecast')
    fig2 = px.line(df, x='ds', y='y', labels={'y': 'Historical Data'}, title=f'{asset_csv} Historical Data')

    fig1.show()
    fig2.show()

# Example usage with an example stock file, adjust the file name as needed
display_forecasts('AAPL.csv', periods=30)




21:57:27 - cmdstanpy - INFO - Chain [1] start processing
21:57:29 - cmdstanpy - INFO - Chain [1] done processing
C:\Users\kahbe\AppData\Local\Temp\ipykernel_10188\166422373.py:26: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



Predicted Values from Today's Date:
           ds        yhat
0  2024-02-09  182.269352
1  2024-02-10  182.139471
2  2024-02-11  182.067799
3  2024-02-12  182.283571
4  2024-02-13  182.056749
5  2024-02-14  181.948307
6  2024-02-15  181.723935
7  2024-02-16  181.473154
8  2024-02-17  181.204357
9  2024-02-18  181.006146
10 2024-02-19  181.109436
11 2024-02-20  180.785630
12 2024-02-21  180.596891
13 2024-02-22  180.309818
14 2024-02-23  180.014562
15 2024-02-24  179.719860
16 2024-02-25  179.514367
17 2024-02-26  179.628775
18 2024-02-27  179.333995
19 2024-02-28  179.191440
20 2024-02-29  178.966732
21 2024-03-01  178.748830
22 2024-03-02  178.545096
23 2024-03-03  178.442656
24 2024-03-04  178.670550
25 2024-03-05  178.497943
26 2024-03-06  178.484443
27 2024-03-07  178.393838
28 2024-03-08  178.313259
29 2024-03-09  178.248274
30 2024-04-21  182.459481
31 2024-04-22  182.759476
32 2024-04-23  182.632382
33 2024-04-24  182.637827
34 2024-04-25  182.539926
35 2024-04-26  182.426432
36